<a href="https://colab.research.google.com/github/ramoants254/Food_101-Deep-Learning-project/blob/main/food_101_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
import random
import zipfile
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers,mixed_precision
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomHeight, RandomWidth, RandomZoom, Rescaling

In [2]:
!wget 'https://storage.googleapis.com/ztm_tf_course/food_vision/06_101_food_class_10_percent_saved_big_dog_model.zip'

--2024-08-20 05:19:57--  https://storage.googleapis.com/ztm_tf_course/food_vision/06_101_food_class_10_percent_saved_big_dog_model.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.210.207, 74.125.26.207, 173.194.212.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.210.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46760742 (45M) [application/zip]
Saving to: ‘06_101_food_class_10_percent_saved_big_dog_model.zip’

06_101_food_class_1 100%[===================>]  44.59M  91.6MB/s    in 0.5s    

2024-08-20 05:19:57 (91.6 MB/s) - ‘06_101_food_class_10_percent_saved_big_dog_model.zip’ saved [46760742/46760742]



In [3]:
!wget 'https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py'

--2024-08-20 05:19:57--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-08-20 05:19:58 (62.4 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Importing Tensorflow Datasets

In [5]:
import tensorflow_datasets as tfds
dataset_list=tfds.list_builders()
print('food101' in dataset_list)

True


In [ ]:
(train_data, test_data), ds_info = tfds.load(name="food101",
                                             split=["train", "validation"],
                                             shuffle_files=True,
                                             as_supervised=True,
                                             with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

## Exploring the Dataset

In [ ]:
ds_info.features

In [ ]:
class_names=ds_info.features['label'].names
class_names[:10]

In [ ]:
train_ones_sample=train_data.take(1)
train_ones_sample

In [ ]:
for image,label in train_ones_sample:
  print(f'Image Shape:{image.shape}')
  print(f'Image Label:{label}')

In [ ]:
image

In [ ]:
plt.imshow(image)
plt.title(class_names[label])
plt.axis(False)

In [ ]:
def preprocess_img(image,label,image_shape=224):
  image=tf.image.resize(image,[image_shape,image_shape])
  # image=image/255.0
  return tf.cast(image,tf.float32),label


In [ ]:
preprocesseded_img=preprocess_img(image,label)[0]
preprocesseded_img

## Batch && Prepare Datasets

In [ ]:
train_data=train_data.map(map_func=preprocess_img,num_parallel_calls=tf.data.AUTOTUNE)
train_data=train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

test_data.map(map_func=preprocess_img,num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
train_data,test_data

## Creating Model Callbacks

In [ ]:
from helper_functions import create_tensorboard_callback
checkpoint_path='model_checkpoints/cp.ckpt.weights.h5'
model_checkpoint=keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 monitor='val_acc',
                                                 save_best_only=True,
                                                 save_weights_only=True,
                                                 verbose=1)

## Mixed precision Training

In [ ]:
mixed_precision.set_global_policy('mixed_float16')

## Building Feature Exatraction

In [ ]:
from logging import logProcesses
input_shape=(224,224,3)
base_model=keras.applications.EfficientNetB0(include_top=False)
base_model.trainable=False

# Creating a functional model
inputs=keras.layers.Input(shape=input_shape,name='input_layer')
x=base_model(inputs,training=False)
x=layers.GlobalAveragePooling2D(name='global_average_pooling_layer')(x)
x=layers.Dense(len(class_names))(x)
outputs=layers.Activation('softmax',dtype=tf.float32,name='softmax_float32')(x)
model=tf.keras.Model(inputs,outputs)

# Compiling the model
model.compile(loss='sparse_categorical_crossentropy',
            optimizer='Adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
  print(layer.name,layer.trainable)

## Fitting the model

In [ ]:
feature_extract=model.fit(train_data,
                          epochs=5,
                          steps_per_epoch=(len(train_data)),
                          validation_data=test_data,
                          validation_steps=int(0.15*len(test_data)),
                          callbacks=[create_tensorboard_callback(dir_name='model_logs',experiment_name='feature_extraction'),model_checkpoint])